In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
! pip install datasets
! ls /content/gdrive/MyDrive/Kegg/ -l
KGdb="/content/gdrive/MyDrive/Kegg"
%cd /content/gdrive/MyDrive/Kegg

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
total 65354472
drwx------ 2 root root        4096 Feb 28 03:58 CTD
drwx------ 2 root root        4096 Feb 26 16:28 download_pubmed
-rw------- 1 root root       33078 Oct 16 14:38 enzyme2genes.csv
-rw------- 1 root root       81795 Feb 28 16:40 identifier.csv
-rw------- 1 root root      114367 Feb 28 15:42 identifiers_s.csv
drwx------ 2 root root        4096 Sep 22 14:08 kgml_output
-rw------- 1 root root       66282 Oct 16 14:39 module2reaction.csv
-rw------- 1 root root       37795 Oct 16 14:39 pathway2module.csv
-rw------- 1 root root 15796004232 Feb 14 21:54 PKG2020S4_A01_Articles.sql
drwx------ 2 root root        4096 Feb 14 23:55 PKG2020S4_A04_Abstract_split
-rw------- 1 root root 29000206940 Feb 12 23:12 PKG202

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import json
import time

In [ ]:
chem_dis = pd.read_csv(os.path.join(KGdb,"CTD/CTD_chemicals_diseases.csv"), comment='#', low_memory=False)
chem_dis["DiseaseID"] = chem_dis["DiseaseID"].str.extract(r'(D\d+)')

chem_names = chem_dis["ChemicalName"].unique()
len(chem_names)

17337

In [ ]:
chem_dis.head()

,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
0,06-Paris-LA-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,D054198,therapeutic,NaN,NaN,NaN,4519131
1,10074-G5,C534883,NaN,Adenocarcinoma,D000230,NaN,MYC,4.07,NaN,26432044
2,10074-G5,C534883,NaN,Adenocarcinoma of Lung,D000077192,NaN,MYC,4.30,NaN,26656844|27602772
3,10074-G5,C534883,NaN,Alopecia,D000505,NaN,AR,4.49,NaN,15902657
4,10074-G5,C534883,NaN,Androgen-Insensitivity Syndrome,D013734,NaN,AR,6.86,300068|312300,1303262|8281139


In [ ]:
pmid_s=[]
for s in chem_dis.PubMedIDs:
  for i in str(s).split("|"):
    pmid_s.append(i.strip())
pmid_s=list(set(pmid_s))
len(pmid_s)

90229

In [ ]:
!pwd

/content/gdrive/MyDrive/Kegg/download_pubmed


In [ ]:
#%cd ./download_pubmed/

# libraries
from datasets import load_dataset
import os
import json

# load all of pubmed -- note that this dataset is 360GB+, so I'm loading this in "streaming mode" -- https://huggingface.co/docs/datasets/stream
# https://huggingface.co/datasets/pubmed
full_pubmed = load_dataset('pubmed', streaming=True)

# make directory if it doesn't exist
if not os.path.isdir('pubmed_portions_rev'):
    os.mkdir('pubmed_portions_rev')

# download pubmed in portions
portion_size = 4000 # 1 mil entries per portion
pubmed_portion = []
counter = 0

# note that I'm only taking the pmid, year, abstract text, abstract title, and authors. There is some more information that you can see by either visiting (https://huggingface.co/datasets/pubmed) or printing one of the entries in the dataset
for idx, entry in enumerate(full_pubmed['train']):

    try:

        # print(entry)
        pmid = str(entry['MedlineCitation']['PMID'])
        if pmid in pmid_s:
            #print(pmid)
            year = entry['MedlineCitation']['DateCompleted']['Year']
            abstract_text = entry['MedlineCitation']['Article']['Abstract']['AbstractText']
            abstract_title = entry['MedlineCitation']['Article']['ArticleTitle']
            abstract_authors_list_L = entry['MedlineCitation']['Article']['AuthorList']['Author']['LastName']
            abstract_authors_list_F = entry['MedlineCitation']['Article']['AuthorList']['Author']['ForeName']
            abstract_authors_list = [abstract_authors_list_L[i] + "." + abstract_authors_list_F[i] for i in range(len(abstract_authors_list_L))]
            pubmed_portion.append({'pmid':pmid, 'year':year, 'abstract_text':abstract_text, 'abstract_title':abstract_title, 'abstract_authors_list':abstract_authors_list})


        if idx % 10000 == 0:
            print('idx', idx, len(pubmed_portion))

        if len(pubmed_portion) % portion_size == 0 and len(pubmed_portion)>0:
            with open(f'./pubmed_portions_rev/pubmed_CTD_{counter}.json', 'w') as fp:
                json.dump(pubmed_portion, fp)
            # reset the list
            pubmed_portion = []
            counter += 1



    except: # any issue with the article
        pass

# save the pubmed portion
with open(f'./pubmed_portions_rev/pubmed_CTD_{counter}.json', 'w') as fp:
    json.dump(pubmed_portion, fp)
# reset the list
pubmed_portion = []
counter += 1

idx 0 0
idx 10000 32
idx 20000 57
idx 30000 97
idx 40000 146
idx 50000 186
idx 60000 234
idx 70000 297
idx 80000 360
idx 90000 415
idx 100000 448
idx 110000 475
idx 120000 500
idx 130000 527
idx 140000 546
idx 150000 574
idx 160000 597
idx 170000 629
idx 180000 653
idx 190000 685
idx 200000 704
idx 210000 730
idx 220000 757
idx 230000 784
idx 240000 814
idx 250000 816
idx 260000 822
idx 270000 846
idx 280000 874
idx 290000 903
idx 300000 917
idx 310000 938
idx 320000 956
idx 330000 970
idx 340000 988
idx 350000 1008
idx 360000 1029
idx 370000 1056
idx 380000 1078
idx 390000 1093
idx 400000 1128
idx 410000 1153
idx 420000 1184
idx 430000 1229
idx 440000 1272
idx 450000 1309
idx 460000 1347
idx 470000 1379
idx 480000 1391
idx 490000 1413
idx 500000 1446
idx 510000 1475
idx 520000 1498
idx 530000 1523
idx 540000 1535
idx 550000 1559
idx 560000 1593
idx 570000 1648
idx 580000 1673
idx 590000 1695
idx 600000 1715
idx 610000 1737
idx 620000 1790
idx 630000 1824
idx 640000 1854
idx 650000 188

In [ ]:
len(pubmed_portion)

213

In [ ]:
with open(f'./pubmed_portions/pubmed_CTD_{counter}.json', 'w') as fp:
                json.dump(pubmed_portion, fp)